In [2]:
%%capture
!pip install wikidataintegrator

In [137]:
from wikidataintegrator import wdi_core, wdi_login 
from getpass import getpass
import pandas as pd
from copy import deepcopy

In [146]:
wbstack = "lux-saf-beta"
wikibase = "https://{}.wiki.opencura.com/".format(wbstack)
api = "https://{}.wiki.opencura.com/w/api.php".format(wbstack)
sparql = "https://{}.wiki.opencura.com/query/sparql".format(wbstack)
entityUri = wikibase.replace("https:", "http:")+"entity/"

WBUSER = "Andrawaag"
WBPASS = getpass()
login = wdi_login.WDLogin(WBUSER, WBPASS, mediawiki_api_url=api)

········


In [147]:
data_load = pd.read_excel("Export_SAF_LUX_10_dead can dance.xlsx", header=0)
localEntityEngine = wdi_core.WDItemEngine.wikibase_item_engine_factory(api,sparql)
pd.set_option('display.max_columns', None)

In [148]:
person_item = localEntityEngine(new_item=True)
person_item.set_label("E21 Person", lang="en")
person_item.write(login)

'Q350'

In [149]:
qid = dict()
query = "PREFIX wdt: <http://{}.wiki.opencura.com/prop/direct/> SELECT ?item ?label WHERE {{?item rdfs:label ?label }}".format(wbstack)
wdi_core.WDItemEngine.execute_sparql_query(query, as_dataframe = True, endpoint=sparql)
for index, row in wdi_core.WDItemEngine.execute_sparql_query(query, as_dataframe = True, endpoint=sparql).iterrows():
    qid[row["label"]] =row["item"].replace(entityUri, "")

In [ ]:
for index, row in data_load.iterrows():
    statements = []
    references = []
    if not pd.isnull(row["Sources of information:Url"]):
        if row["Sources of information:Url"].startswith("http"):
            references.append(wdi_core.WDUrl(value=row["Sources of information:Url"], prop_nr=qid["source of information - url"], is_reference=True))
            references.append(wdi_core.WDString(value=row["Sources of information:Text"], prop_nr=qid["source of information - text"], is_reference=True))
            # is a person 
            statements.append(wdi_core.WDItemID(value=qid["E21 Person"], prop_nr=qid['instance of']))
            # name qualifiers
            qualifiers_name =[]
            ## type
            if row["Heading:Type"] == "1": 
                qualifiers_name.append(wdi_core.WDItemID(value=qid["indirect name form"], prop_nr=qid["name format"], is_qualifier=True))
            elif row["Heading:Type"] == "0":
                qualifiers_name.append(wdi_core.WDItemID(value=qid["direct name form"], prop_nr=qid["name format"], is_qualifier=True))
            ## numeration 
            if not pd.isnull(row["Heading:Numeration"]):
                qualifiers_name.append(wdi_core.WDString(value=row["Heading:Numeration"], prop_nr=qid["numeration"], is_qualifier=True))
            ## title
            if not pd.isnull(row["Heading:Title"]):
                qualifiers_name.append(wdi_core.WDString(value=row["Heading:Title"], prop_nr=qid["title"], is_qualifier=True))
            # add name 
            statements.append(wdi_core.WDString(value=row["Heading:Name"], prop_nr=qid["name"], qualifiers=deepcopy(qualifiers_name), references=[deepcopy(references)]))     

            if not pd.isnull(row["Alternative / other form(s) of name:Name"]):
                alternative_name_statement = []
                alternative_names = row["Alternative / other form(s) of name:Name"].split(";")
                print(row["Alternative / other form(s) of name:Type"])
                an_types = str(row["Alternative / other form(s) of name:Type"]).split(";")
                if not pd.isnull(row["Alternative / other form(s) of name:Numeration"]):
                    an_numerations = row["Alternative / other form(s) of name:Numeration"].split(";")
                else: 
                    an_numerations = None
                if not pd.isnull(row["Alternative / other form(s) of name:Title"]):
                    an_titles = row["Alternative / other form(s) of name:Title"].split(";")
                    print(an_titles)
                else:
                    an_titles = dict()
                index = 0
                for an in alternative_names:
                    qualifiers_aname = []

                    ## type
                    if len(an_types)>0:
                        if index < len(an_types):
                            if an_types[index] == "1":
                                qualifiers_aname.append(wdi_core.WDItemID(value=qid["indirect name form"], prop_nr=qid["name format"], is_qualifier=True)) 
                            elif an_types[index] == "0":
                                qualifiers_aname.append(wdi_core.WDItemID(value=qid["direct name form"], prop_nr=qid["name format"], is_qualifier=True)) 
                    ## numeration 
                    if not pd.isnull(an_numerations):
                        if index in an_numerastions.keys():
                            qualifiers_aname.append(wdi_core.WDString(value=an_numeration[index], prop_nr=qid["numeration"], is_qualifier=True))

                    if len(an_titles)>0:
                        if index < len(an_titles):
                            qualifiers_aname.append(wdi_core.WDString(value=an_titles[index], prop_nr=qid["title"], is_qualifier=True))
                    # add name 
                    statements.append(wdi_core.WDString(value=an, prop_nr=qid["alternative name"], qualifiers=deepcopy(qualifiers_aname), references=[deepcopy(references)]))
                    index +=1
            
            #gender
            if not pd.isnull(row["Gender:Gender"]): 
                if str(int(row["Gender:Gender"])) == "1":
                    statements.append(wdi_core.WDItemID(value=qid["male"], prop_nr=qid["gender"], references=[deepcopy(references)]))
                elif str(int(row["Gender:Gender"])) == "2":
                    statements.append(wdi_core.WDItemID(value=qid["female"], prop_nr=qid["gender"], references=[deepcopy(references)]))
                elif str(int(row["Gender:Gender"])) == "0":
                    statements.append(wdi_core.WDItemID(value=qid["not known"], prop_nr=qid["gender"], references=[deepcopy(references)]))
                elif str(int(row["Gender:Gender"])) == "9":
                    statements.append(wdi_core.WDItemID(value=qid["not applicable"], prop_nr=qid["gender"], references=[deepcopy(references)]))

            #birthdate
            #statements.append(wdi_core.WDEdtf(value=row["Associated identification information:Birth date"], prop_nr=qid["date of birth"]))
            #birthplace
            if not pd.isnull(row["Associated identification information:Birth place"]):
                statements.append(wdi_core.WDString(value=row["Associated identification information:Birth place"], prop_nr=qid["place of birth"], references=[deepcopy(references)]))

            # place of death
            if not pd.isnull(row["Associated identification information:Death place"]):
                statements.append(wdi_core.WDString(value=row["Associated identification information:Death place"], prop_nr=qid["place of death"], references=[deepcopy(references)]))
            
            # profession
            if not pd.isnull(row["Profession:Type"]):
                statements.append(wdi_core.WDString(value=row["Profession:Type"], prop_nr=qid["profession"], references=[deepcopy(references)]))

            if not pd.isnull(row["Activity:Type"]):
                activities = row["Activity:Type"].split(";")
            
            if not pd.isnull(row["Activity:Beginning"]):
                activities_start = str(row["Activity:Beginning"]).split(";")
                
            if not pd.isnull(row["Activity:End"]):
                activities_start = str(row["Activity:End"]).split(";")

            j = 0
            for activity in activities:
                statements.append(wdi_core.WDString(value=activity, prop_nr=qid["activity"], references=[deepcopy(references)]))
                # TODO add qualifiers once EDTF is supported   
                
            # internal identifier
            statements.append(wdi_core.WDExternalID(value=str(row["Administration fields:Internal standard identifier"]), prop_nr=qid["AFL identifier"], references=[deepcopy(references)]))

            # external identifiers
            identifiers = str(row["Administration fields:External standard identifier:ID"]).split(";")
            sources = str(row["Administration fields:External standard identifier:Name"]).split(";")
            if len(identifiers) != len(sources):
                print("ERROR: number of identifiers do not match number of sources")
                continue
            i = 0
            for identifier in identifiers:
                if sources[i] == "ARK":
                    statements.append(wdi_core.WDUrl(value=identifier, prop_nr=qid["ARK"], references=[deepcopy(references)]))
                elif sources[i] == "VIAF":
                    statements.append(wdi_core.WDExternalID(value=identifier, prop_nr=qid["VIAF"], references=[deepcopy(references)]))
                elif sources[i] == "ISNI":
                    statements.append(wdi_core.WDExternalID(value=identifier, prop_nr=qid["ISNI"], references=[deepcopy(references)]))
                print(identifier, sources[i])
                i +=1
            print(row["Heading:Name"])
            if row["Heading:Name"] not in qid.keys():
                item = localEntityEngine(new_item=True, data=statements)
            else:
                item = localEntityEngine(wd_item_id=qid[row["Heading:Name"]], data=statements)

            item.set_label(row["Heading:Name"], lang="lb")
            item.set_label(row["Heading:Name"], lang="de")
            item.set_label(row["Heading:Name"], lang="en")
            item.set_label(row["Heading:Name"], lang="fr")
            qid[row["Heading:Name"]] = item.write(login)
            print(row["Heading:Name"], qid[row["Heading:Name"]])
        else:
            print(row["Heading:Name"], "url does not contain header")


Zorn, Wilhelm url does not contain header
0;1;0;0;1
0000 0000 4390 2515 ISNI
https://persist.lu/ark:70795/mb1387x74 ARK
77038289 VIAF
Zenner, Theodor
Zenner, Theodor Q351
0;1
https://persist.lu/ark:70795/nc0kv6rqb ARK
15145003276461300298 VIAF
Zenner, Jean
Zenner, Jean Q352
Zanussi, Marc url does not contain header
0
0000 0003 8517 1599 ISNI
https://persist.lu/ark:70795/0153psjrq ARK
35462746 VIAF
Zanen, Jean-Pierre
Zanen, Jean-Pierre Q353
0;1
https://persist.lu/ark:70795/p87fq64br ARK
9779161098987429640006 VIAF
Wust, Jean-Pierre
Wust, Jean-Pierre Q354
1;1;1
https://persist.lu/ark:70795/mpm2fwdfb ARK
56145003829861340757 VIAF
Wurth, Paul
Wurth, Paul Q355
https://persist.lu/ark:70795/ckcvrfqf8 ARK
60145003278961300941 VIAF
Wurth, Marcel
Wurth, Marcel Q356
Wurth, Ernest url does not contain header
Wolter, Jean url does not contain header
Wolff, Johann-Georg url does not contain header
1
0000 0000 1500 5358 ISNI
https://persist.lu/ark:70795/04x1xb0s6 ARK
72411882 VIAF
Witry, Théo
Witry, 

Schlechter, Demy Q412
0
https://persist.lu/ark:70795/tk32p3xfm ARK
63145003277761300106 VIAF
Schiltz, Roger
Schiltz, Roger Q413
0;1
https://persist.lu/ark:70795/mfnzb0ndj ARK
72145003276461301653 VIAF
Scherff, Georges de
Scherff, Georges de Q414
0;1
https://persist.lu/ark:70795/qvwcj2k44 ARK
315731185 VIAF
Schaus, Raymond
Schaus, Raymond Q415
0;1
0000 0001 3945 6468 ISNI
https://persist.lu/ark:70795/r4pw6x3kb ARK
193091483 VIAF
Schaus, Lambert
Schaus, Lambert Q416
https://persist.lu/ark:70795/g2r4m0z4b ARK
73145003277261301475 VIAF
Schaus, Jean-Nicolas
Schaus, Jean-Nicolas Q417
59145003280961298432 VIAF
Schaeffer, Marcel
Schaeffer, Marcel Q418
https://persist.lu/ark:70795/8g1v56ntr ARK
4161096227729100392 VIAF
Schaefer, Charles
Schaefer, Charles Q419
0000 0000 1427 5380 ISNI
311421990 VIAF
311421990 VIAF
Schaack, Lucien
Schaack, Lucien Q420
https://persist.lu/ark:70795/rvpmxgz6n ARK
9161096559029101930 VIAF
Ruppert, Jules Paul
Ruppert, Jules Paul Q421
Rischette, Arthur url does not con

Loesch, Jacques Q481
1;1;1;1;1;1
0000 0000 6300 2661 ISNI
37423261 VIAF
Loenertz, Raymond-Joseph
Loenertz, Raymond-Joseph Q482
Linden, Johann url does not contain header
1;0
0000 0003 9138 1066 ISNI
239321349 VIAF
Leydenbach, Joseph
Leydenbach, Joseph Q483
1;1;1;1;1;0;1
ERROR: number of identifiers do not match number of sources
1;1
0000 0000 2002 1440 ISNI
77452174 VIAF
Leir, Henry J.
Leir, Henry J. Q484
1
2422161098964820033536 VIAF
Lefort, Émile
Lefort, Émile Q485
1;1
nan nan
Lefèvre, Nina
Lefèvre, Nina Q486
0
0000 0001 0303 8635 ISNI
https://persist.lu/ark:70795/rh6xvxsk8 ARK
153851853 VIAF
Leclère, René
Leclère, René Q487
1
0000 0000 3890 8891 ISNI
https://persist.lu/ark:70795/mt76bkrc9 ARK
58558589 VIAF
Le Jeune, Jules
Le Jeune, Jules Q488
Laurent, François url does not contain header
https://persist.lu/ark:70795/9f7t5mxjb ARK
4161094535629100132 VIAF
Lauer, Léo
Lauer, Léo Q489
Latomus, Bartholomaeus url does not contain header
0000 0000 2993 1897 ISNI
76876786 VIAF
Lascombes, Fr

In [129]:
data_load

,Heading:Name,Heading:Type,Heading:Numeration,Heading:Title,Alternative / other form(s) of name:Name,Alternative / other form(s) of name:Type,Alternative / other form(s) of name:Numeration,Alternative / other form(s) of name:Title,Associated identification information:Birth date,Associated identification information:Birth place,Associated identification information:Death date,Associated identification information:Death place,Gender:Gender,Profession:Type,Profession:Beginning,Profession:End,Activity:Type,Activity:Beginning,Activity:End,Sources of information:Text,Sources of information:Url,Administration fields:External standard identifier:ID,Administration fields:External standard identifier:Name,Administration fields:Internal standard identifier,Administration fields:Source of data (Institution codes),Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29
0,"Zorn, Wilhelm",1,NaN,NaN,"Zorn, Guillaume",1,NaN,NaN,1850-06-14,Ell,1920-01-24 00:00:00,Saeul,1.0,Ecclésiastique,NaN,NaN,Auteur,NaN,NaN,"Blum, Martin. Bibliographie luxembourgeoise, 1...",www.a-z.lu/BIBNET:ALEPH_LUX01000053630,https://persist.lu/ark:70795/krx4xjzfc;7161096...,ARK;VIAF,48613,LU-BNL,NaN,NaN,NaN,NaN,NaN
1,"Zenner, Theodor",1,NaN,NaN,"R.R.;Zenner, Théodore;Der Kalenderonkel;Th.Z.;...",0;1;0;0;1,NaN,NaN,1890-02-22,Gilsdorf,1948-04-26 00:00:00,Ospern,1.0,NaN,NaN,NaN,Auteur;Prêtre;Curé à Folscheid;Doyen à Ospern,1914;1926;1935,1935;1948,Dictionnaire des auteurs luxembourgeois,http://www.autorenlexikon.lu/page/author/306/3...,0000 0000 4390 2515;https://persist.lu/ark:707...,ISNI;ARK;VIAF,10126,LU-BNL,NaN,NaN,NaN,NaN,NaN
2,"Zenner, Jean",1,NaN,NaN,"N.R.;Zenner, Johann Nicolas",0;1,NaN,NaN,1900-04-16 00:00:00,Gilsdorf,1954-05-10 00:00:00,Gilsdorf,1.0,NaN,NaN,NaN,Auteur;Déporté au camp de concentration de Hin...,NaN,NaN,Dictionnaire des auteurs luxembourgeois,http://www.autorenlexikon.lu/page/author/381/3...,https://persist.lu/ark:70795/nc0kv6rqb;1514500...,ARK;VIAF,10117,LU-BNL,NaN,NaN,NaN,NaN,NaN
3,"Zanussi, Marc",1,NaN,NaN,NaN,NaN,NaN,NaN,1959-03-11 00:00:00,Dudelange,2004-06-24 00:00:00,Wissant,1.0,Homme politique,NaN,NaN,Sous-chef de brigade aux CFL;Président de l'as...,1978;1990;1991;2000,1986;2004;2004;2004,"Bodry, Alex, Fayot, Ben. 120 Jor sozialistesch...",www.a-z.lu/BIBNET:ALEPH_LUX01001456372;www.a-z...,https://persist.lu/ark:70795/chqtpq4cn;7234161...,ARK;VIAF,47422,LU-BNL,NaN,NaN,NaN,NaN,NaN
4,"Zanen, Jean-Pierre",1,NaN,NaN,J. vun der Hardt,0,NaN,NaN,1878-12-12,Weicherdange,1953-06-17 00:00:00,Luxembourg,1.0,Professeur de gymnase,NaN,NaN,Auteur;Professeur au Lycée technique agricole;...,1912,NaN,Dictionnaire des auteurs luxembourgeois;lb.wik...,http://www.autorenlexikon.lu/page/author/283/2...,0000 0003 8517 1599;https://persist.lu/ark:707...,ISNI;ARK;VIAF,5394,LU-BSL,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
971,"Adam, Mathias",1,NaN,NaN,NaN,NaN,NaN,NaN,1850-11-19,Luxembourg-Grund,1936-02-17 00:00:00,Mondorf-les-Bains,1.0,Enseignant,NaN,NaN,Instituteur à l'école primaire supérieure de P...,1887;1900;1883;1901,1915;1918;1901,"lb.wikipedia.org, consulté le 23.03.2016;Molit...",https://lb.wikipedia.org/wiki/Mathias_Adam;www...,https://persist.lu/ark:70795/jrtcvw1tc;1676161...,ARK;VIAF,45853,LU-BNL,NaN,NaN,NaN,NaN,NaN
972,"Adam, Joseph",1,NaN,NaN,J.A.,0,NaN,NaN,1918-10-06 00:00:00,Bissen,2000-09-02 00:00:00,Luxembourg-ville,1.0,Ecclésiastique,NaN,NaN,Auteur;Prêtre;Recteur au de la Communauté jésu...,1944;1947;1960;1956,1950;1991;1986,"Schumacher, Roland. Vikare der Pfarrei Hesperi...",www.a-z.lu/BIBNET:ALEPH_LUX01000343571;https:/...,0000 0000 3212 0438;311151778,ISNI;VIAF,54265,LU-BNL,NaN,NaN,NaN,NaN,NaN
973,"Adam, Jean-Pierre",1,NaN,NaN,"Adam, Menn",1,NaN,NaN,1941-08-05 00:00:00,Dudelange,2014-01-13 00:00:00,Dudelange,1.0,Artiste,NaN,NaN,Artiste-peintre;Graveur;Sculpteur;Editeur du l...,1972;1973,NaN,"lb.wikipedia.org, consulté le 05.08.2016;Hause.

In [56]:
tjoepie = data_load[data_load["Heading:Name"] =="Willibrordus"]

In [112]:
for index, row in data_load.iterrows():
    print()

1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1


ValueError: cannot convert float NaN to integer

In [134]:
item.get_wd_json_representation()

{'claims': {'P1': [{'mainsnak': {'snaktype': 'value',
     'property': 'P1',
     'datavalue': {'value': {'entity-type': 'item',
       'numeric-id': 17,
       'id': 'Q17'},
      'type': 'wikibase-entityid'},
     'datatype': 'wikibase-item'},
    'type': 'statement',
    'rank': 'normal',
    'qualifiers': {},
    'qualifiers-order': [],
    'references': []}],
  'P17': [{'mainsnak': {'snaktype': 'value',
     'property': 'P17',
     'datavalue': {'value': 'R.R.', 'type': 'string'},
     'datatype': 'string'},
    'type': 'statement',
    'rank': 'normal',
    'qualifiers': {'P9': [{'snaktype': 'value',
       'property': 'P9',
       'datavalue': {'value': {'entity-type': 'item',
         'numeric-id': 15,
         'id': 'Q15'},
        'type': 'wikibase-entityid'},
       'datatype': 'wikibase-item'}]},
    'qualifiers-order': ['P9'],
    'references': [{'snaks': {'P16': [{'snaktype': 'value',
         'property': 'P16',
         'datavalue': {'value': 'http://www.autorenlexikon.l